<a href="https://colab.research.google.com/github/annabocharovaua/Scraping_service/blob/main/JobHuntHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telebot > null
!pip install selenium > null
!apt-get update > null
!apt-get install -y chromium-chromedriver > null
!pip install mysql-connector-python > null


In [2]:
import telebot
from telebot import types
from enum import Enum
from urllib.parse import urlencode
from selenium import webdriver
import mysql.connector
from threading import Lock

In [3]:
# Создать пул подключений с заданным размером
connection_pool = mysql.connector.pooling.MySQLConnectionPool(
    pool_name="my_pool",
    pool_size=5,  # Задать размер пула подключений
    host="4.tcp.eu.ngrok.io",
    port=16590,
    user="root",
    password="root",
    database="scraping_service"
)

import mysql.connector

# Функция для получения подключения из пула
def get_connection():
    return connection_pool.get_connection()

# Мьютекс для синхронизации доступа к функции выполнения запросов
query_lock = Lock()

# Выполнение запросов
def execute_query(query, params=None):
  with query_lock:
    connection = get_connection()
    cursor = connection.cursor()
    try:
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)

        # Если запрос предполагает возвращение данных (SELECT), получаем результаты
        if query.strip().upper().startswith('SELECT'):
            results = cursor.fetchall()
        else:
            # Для запросов, модифицирующих данные (INSERT, UPDATE, DELETE),
            # сохраняем изменения и возвращаем None
            connection.commit()
            results = []

    except Exception as e:
        # В случае ошибки выводим ее и возвращаем None
        print(f"Ошибка при выполнении запроса: {e}")
        results = None

    cursor.close()
    connection.close()
    return results

# Пример использования
result = execute_query("SELECT * FROM users")
if result:
    for row in result:
        print(row)
else:
    print("Не удалось выполнить запрос.")


#print("AAAA: ", execute_query(f"INSERT INTO users (chat_id, paid_subscription) VALUES ({10}, 0);"))

(2, 2, 1, 0, None)
(3, 3, 1, 0, None)
(10, 0, 1, 0, None)
(535434829, 0, 0, 3, None)
(869704769, 2, 4, 5, '2024-6-3')


In [4]:
from urllib.parse import urlencode, quote

class EXPERIENCE(Enum):
  DEFAULT_VALUE = -2
  STUDENT = -1
  WITHOUT_EXPERIENCE = 0
  ONE_YEAR = 1
  TWO_YEAR = 2
  THREE_YEAR = 3
  FOUR_YEAR = 4
  FIVE_YEAR = 5

def build_url(base_url, **params):
  # Видаляємо всі елементи, які мають None або порожнє значення
  filtered_params = {k: v for k, v in params.items() if v}
  query_string = urlencode(filtered_params)
  return f"{base_url}?{query_string}" if query_string else base_url

def build_url_djinni(**params):
  base_url = "https://djinni.co/jobs/"

  params["location"] = params["location"].lower()
  if params["location"] == "remote":
    params["employment"]="remote"
    del params["location"]

  if params["exp_level"] == EXPERIENCE.DEFAULT_VALUE.value or params["exp_level"] > EXPERIENCE.FIVE_YEAR.value:
     params["exp_level"] = ""
  else:
     params["exp_level"] = str(params["exp_level"]) + 'y'
  if params["exp_level"] == "0y":
     params["exp_level"] = "no_exp"

  params["exp_rank"] = params["exp_rank"].lower()
  return build_url(base_url, **params)

def build_url_workua(language='', city='', position='', min_salary=0, experience=EXPERIENCE.DEFAULT_VALUE.value, **params):
  base_url = "https://www.work.ua/jobs"
  if city:
    base_url += '-' + city
  if language:
    base_url += '-' + quote(language)
  if position:
    base_url += '+' + position
  base_url += '/'
  salaryidfrom=''
  if min_salary:
    if 3000 <= min_salary and min_salary < 5000:
      params["salaryfrom"] = 2
    elif 5000 <= min_salary and min_salary < 7000:
      params["salaryfrom"] = 3
    elif 7000 <= min_salary and min_salary < 10000:
      params["salaryfrom"] = 4
    elif 10000 <= min_salary and min_salary < 15000:
      params["salaryfrom"] = 5
    elif 15000 <= min_salary and min_salary < 20000:
      params["salaryfrom"] = 6
    elif 20000 <= min_salary and min_salary < 30000:
      params["salaryfrom"] = 7
    elif min_salary > 30000:
      params["salaryfrom"] = 8

  if experience == EXPERIENCE.WITHOUT_EXPERIENCE.value:
    params["experience"] = 1
  elif experience == EXPERIENCE.STUDENT.value:
    params["student"] = 1

  return build_url(base_url, **params)


def convert_uah_to_usd_for_djinni(uah_amount, exchange_rate):
  # Визначаємо список значень для округлення
  round_values = [1500, 2500, 3500, 4500, 5500, 6500, 7500, 8500]
  usd_amount = uah_amount / exchange_rate
  if (usd_amount < 1500):
    return ''
  rounded_down_value = max([value for value in round_values if value <= usd_amount], default=min(round_values))
  return rounded_down_value

def build_url_dou(**params):
  base_url = "https://jobs.dou.ua/vacancies/"
  if params["city"] == "remote":
      del params["city"]
      params["remote"] = 'remote'

  if params.get("position"):
      params["search"] += " " + params["position"]
      del params["position"]
  if params["exp"] == EXPERIENCE.STUDENT.value or params["exp"] == EXPERIENCE.WITHOUT_EXPERIENCE.value or params["exp"] == EXPERIENCE.ONE_YEAR.value:
     params["exp"] = "0-1"
  elif params["exp"] == EXPERIENCE.TWO_YEAR.value or params["exp"] == EXPERIENCE.THREE_YEAR.value:
     params["exp"] = "1-3"
  elif params["exp"] == EXPERIENCE.FOUR_YEAR.value or params["exp"] == EXPERIENCE.FIVE_YEAR.value:
     params["exp"] = "3-5"
  elif params["exp"] > EXPERIENCE.FIVE_YEAR.value:
     params["exp"] = "5plus"
  else:
    del params["exp"]

  return build_url(base_url, **params)

def build_url_robota(language='', city='', **params):
  base_url = "https://robota.ua/zapros/"
  if language:
      base_url += quote(language).lower()
  if params["position"]:
      base_url += '-' + params["position"].lower()
      del params["position"]
  base_url += '/'
  if city == "remote":
      params["scheduleIds"] = 3
  elif city:
      base_url += city.lower() + '/'

  if params["experience"] == EXPERIENCE.WITHOUT_EXPERIENCE.value:
     params["experienceType"] = "true"
  params["experience"] = ""

  return build_url(base_url, **params)


def build_url_jooble(**params):
  base_url = "https://ua.jooble.org/SearchResult"

  if params["workExp"] == EXPERIENCE.STUDENT.value or params["workExp"] == EXPERIENCE.WITHOUT_EXPERIENCE.value:
    params["workExp"] = 1
  else:
    params["workExp"] = ""

  if params["rgns"] == "remote":
    params["loc"]=2
    params["rgns"] = ""
  return build_url(base_url, **params)


url = build_url_robota(
    language="c++",
    city="kyiv",
    position="junior",
    salary=20340,
    experience=0
)
print(url)

url = build_url_workua(
    language="c++",
    city="kyiv",
    #position="junior",
    #min_salary=20340,
    #experience=0,
    days=122
)
print(url)


url = build_url_djinni(
    all_keywords="python",
    primary_keyword="c#",
    region="UKR",
    location="Kyiv",
    exp_level=-2,
    exp_rank="junior",
    salary=convert_uah_to_usd_for_djinni(20, 39.1),
    keywords="c++"
)
print(url)

url = build_url_dou(city="remote", search="c++", position="junior", exp=1) #в доу можна скрапить і по 0-1 і по 1-3
print(url)

jooble_url = build_url_jooble(date=3, #щоб вакансії за останню неділлю
                              rgns="remote",
                              salaryMin="1000",
                              ukw="junior" + ' ' + "c++",
                              workExp=0
                              )
print(jooble_url)

https://robota.ua/zapros/c%2b%2b-junior/kyiv/?salary=20340&experienceType=true
https://www.work.ua/jobs-kyiv-c%2B%2B/?days=122
https://djinni.co/jobs/?all_keywords=python&primary_keyword=c%23&region=UKR&location=kyiv&exp_rank=junior&keywords=c%2B%2B
https://jobs.dou.ua/vacancies/?search=c%2B%2B+junior&exp=0-1&remote=remote
https://ua.jooble.org/SearchResult?date=3&salaryMin=1000&ukw=junior+c%2B%2B&workExp=1&loc=2


In [5]:
import requests
import codecs
from bs4 import BeautifulSoup as BS
from random import randint
from urllib.parse import urlencode
import re


__all__ = ('work', "robota", 'dou', 'djinni')

headers = [
    {'User-Agent': 'Mozilla/5.0 (Windows NT 5.1; rv:47.0) Gecko/20100101 Firefox/47.0',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; rv:53.0) Gecko/20100101 Firefox/53.0',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
    ]



def work(url, city=None, language=None):
    jobs = []
    errors = []
    domain = 'https://www.work.ua'
    if url:
        resp = requests.get(url, headers=headers[randint(0, 2)])
        if resp.status_code == 200:
            soup = BS(resp.content, 'html.parser')
            main_div = soup.find('div', id='pjax-job-list')
            if main_div:
                div_lst = main_div.find_all('div', attrs={'class': 'job-link'})
                # ігноруємо вакансії після p з текстом Вакансії, пов’язані із запитом «python junior» у Києві
                # Ініціалізація списку для зберігання дівів, які знаходяться перед вказаним тегом <p>
                target_p = soup.find('p', class_='text-default-7 add-bottom')

                result_divs = []
                if target_p:
                  for div in target_p.find_all_previous('div', class_='job-link'):
                      result_divs.append(div)
                else:
                  result_divs = div_lst


                for div in result_divs:
                    title = div.find('h2')
                    href = title.a['href']
                    content = div.p.text
                    description = re.sub(r'\s+', ' ', content).strip()
                    img_div = div.find('div', class_ = 'add-bottom')

                    spans = div.find_all('span',  class_='strong-600')
                    salary = '0'
                    company = ''
                    if len(spans)==2:
                      salary = spans[0].text
                      company = spans[1].text
                    else:
                      company = spans[0].text

                    if img_div:
                      img_tag = img_div.find('img')
                      if img_tag:  # Якщо тег <img> присутній
                          img_src = img_tag['src']
                      else:
                          img_src = 'https://st.work.ua/i/work-ua-knowledge-graph.jpg'

                    jobs.append({'title': title.text, 'img_source': img_src, 'url': domain + href,
                                 'description': description, 'company': company,
                                 'city_id': city, 'language_id': language,'salary': salary, 'site': "workua"})
            else:
                errors.append({'url': url, 'title': "Div does not exists"})
        else:
            errors.append({'url': url, 'title': "Page do not response"})

    return jobs, errors

In [6]:
def dou_image(url):
    image_src = ''
    if url:
        resp = requests.get(url, headers=headers[randint(0, 2)])
        if resp.status_code == 200:
            soup = BS(resp.content, 'html.parser')
            comp_div = soup.find('div', attrs={'class': 'b-compinfo'})
            if comp_div:
                a = comp_div.find('a', attrs={'class': 'logo'})
                image_src = a.img['src']

    return image_src


def dou(url, city=None, language=None):
    jobs = []
    errors = []

    if url:
        resp = requests.get(url, headers=headers[randint(0, 2)])
        if resp.status_code == 200:
            soup = BS(resp.content, 'html.parser')
            main_div = soup.find('div', id='vacancyListId')
            if main_div:
                li_lst = main_div.find_all('li', attrs={'class': 'l-vacancy'})
                for li in li_lst:
                    title = li.find('div', attrs={'class': 'title'})
                    href = title.a['href']
                    cont = li.find('div', attrs={'class': 'sh-info'})
                    content = cont.text
                    company = 'No name'
                    salary = '0'
                    a = title.find('a', attrs={'class': 'company'})
                    img_src = dou_image(href)
                    if a:
                        company = a.text
                    jobs.append({'title': title.text, 'img_source': img_src, 'url': href,
                                 'description': content, 'company': company,
                                 'city_id': city, 'language_id': language, 'salary': salary, 'site': "dou"})
            else:
                errors.append({'url': url, 'title': "Div does not exists"})
        else:
            errors.append({'url': url, 'title': "Page do not response"})

    return jobs, errors



In [7]:
def djinni(url, city=None, language=None):
    jobs = []
    errors = []
    domain = 'https://djinni.co'
    if url:
        resp = requests.get(url, headers=headers[randint(0, 2)])
        if resp.status_code == 200:
            soup = BS(resp.content, 'html.parser')
            main_ul = soup.find('ul',  attrs={'class': 'list-unstyled list-jobs mb-4'})
            if main_ul:
                li_lst = main_ul.find_all('li',
                                          attrs={'class': 'list-jobs__item'})
                for li in li_lst:
                    title_div = li.find('div',
                                    attrs={'class': 'job-list-item__title'})
                    if title_div:
                        title = title_div.a
                        href = title['href']

                    else:
                        print("Div з класом 'job-list-item__title' не знайдено.")

                    cont = li.find('div', attrs={'class': 'job-list-item__description'})
                    span = cont.find('span')
                    content = span.text
                    company = 'No name'
                    span_salary = li.find('span', attrs={'class': 'public-salary-item'})
                    if span_salary:
                        salary = span_salary.text
                    else:
                        salary='0'
                    comp = li.find('a', attrs={'class': 'mr-2'})

                    img_div = li.find('div', attrs={'class': 'userpic-wrapper userpic-color_2 userpic_xs userpic-transparent userpic--logo'})
                    if img_div:
                        img_src = img_div.img['src']
                    else:
                        img_src = 'https://sourcingsummit.net/sosutech/wp-content/uploads/sites/30/2017/07/djinnix505x235-1.jpg'

                    if comp:
                        company = comp.text

                    jobs.append({'title': title.text, 'img_source': img_src, 'url': domain+href,
                                 'description': content, 'company': company,
                                 'city_id': city, 'language_id': language, 'salary': salary, 'site': "djinni"})
            else:
                errors.append({'url': url, 'title': "Div does not exists"})
        else:
            errors.append({'url': url, 'title': "Page do not response"})

    return jobs, errors

In [8]:
def helper_robota(url):
    jobs = []
    errors = []
    domain = 'https://djinni.co'
    if url:
        resp = requests.get(url, headers=headers[randint(0, 2)])
        if resp.status_code == 200:
            soup = BS(resp.content, 'html.parser')
            main_ul = soup.find('ul',  attrs={'class': 'list-unstyled list-jobs mb-4'})
            if main_ul:
                li_lst = main_ul.find_all('li',
                                          attrs={'class': 'list-jobs__item'})
                for li in li_lst:
                    title_div = li.find('div',
                                    attrs={'class': 'job-list-item__title'})
                    if title_div:
                        title = title_div.a
                        href = title['href']

                    else:
                        print("Div з класом 'job-list-item__title' не знайдено.")

                    cont = li.find('div', attrs={'class': 'job-list-item__description'})
                    span = cont.find('span')
                    content = span.text
                    company = 'No name'
                    span_salary = li.find('span', attrs={'class': 'public-salary-item'})
                    if span_salary:
                        salary = span_salary.text
                    else:
                        salary='0'
                    comp = li.find('a', attrs={'class': 'mr-2'})

                    img_div = li.find('div', attrs={'class': 'userpic-wrapper userpic-color_2 userpic_xs userpic-transparent userpic--logo'})
                    if img_div:
                        img_src = img_div.img['src']
                    else:
                        img_src = 'https://sourcingsummit.net/sosutech/wp-content/uploads/sites/30/2017/07/djinnix505x235-1.jpg'

                    if comp:
                        company = comp.text

                    jobs.append({'title': title.text, 'img_source': img_src, 'url': domain+href,
                                 'description': content, 'company': company,
                                 'city_id': city, 'language_id': language, 'salary': salary, 'site': "robota"})
            else:
                errors.append({'url': url, 'title': "Div does not exists"})
        else:
            errors.append({'url': url, 'title': "Page do not response"})

    return jobs, errors

In [9]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

def robota(url, city=None, language=None):
    jobs = []
    errors = []
    domain = 'https://robota.ua'

    # Инициализация драйвера Chrome
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.binary_location = 'chromedriver.exe'
    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get(url)

        for i in range(10):
          driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
          time.sleep(2)  # Затримка між прокрутками

        # Затримка для завантаження додаткового вмісту
        time.sleep(5)

        # Отримання HTML-вмісту сторінки
        page_source = driver.page_source


        soup = BS(page_source, 'html.parser')

        # Поиск блоков с вакансиями
        job_divs = soup.find_all('alliance-vacancy-card-desktop')
        for job_div in job_divs:
            #print(job_div)
            a = job_div.find('a', class_='card')
            if a:
                href = a['href']
            else:
                href = ''

            title_tag = job_div.find('h2', class_='santa-typo-h3')
            if title_tag:
                title = title_tag.text.strip()
            else:
                title = ''

            spans = job_div.find_all('span', class_='ng-trigger')
            salary = '0'
            company = ''
            for span in spans:
                text = span.text.strip()
                if '₴' in text:  # Если в тексте есть символ валюты, предполагаем, что это зарплата
                    salary = text
                else:
                    company = text
                    break
            # изображение блокируется на сайте работа
            img_div = job_div.find('div', attrs={'class': 'company-logo'})
            img_src = 'https://play-lh.googleusercontent.com/mr-SZKDW5xmXOinA1cQWz6VeQ10BMKS_UfjGpSxrtoRAl9tNe6HPq5ALFCBAQE2Gw8A'
            if img_div:
                img_tag = img_div.find('img')
                if img_tag:
                    img_src = img_tag['src']

            content_divs = job_div.find_all('div', attrs={'class': 'badge'})
            content = ''
            if content_divs:
                for div in content_divs :
                    content += div.text + "."
                content += ".."

            jobs.append({'title': title, 'img_source': img_src, 'url': domain + href,
                         'description': content, 'company': company,
                         'city_id': city, 'language_id': language, 'salary': salary, 'site': "robota"})

    except Exception as e:
        errors.append({'url': url, 'title': str(e)})

    finally:
        # Закрытие браузера
        driver.quit()

    return jobs, errors

# url = 'https://robota.ua/zapros/java-junior/kyiv'
# print(url)
# jobs, errors = robota(url)
# print(jobs)
# print(errors)

In [10]:
def jooble(url):
    jobs = []
    errors = []
    if url:
        resp = requests.get(url, headers=headers[randint(0, 2)])
        if resp.status_code == 200:
            soup = BS(resp.content, 'html.parser')
            main_div = soup.find('div', class_='infinite-scroll-component ZbPfXY _serpContentBlock')
            if main_div:
                div_lst = main_div.find_all('div', attrs={'data-test-name': '_jobCard'})

                for div in div_lst:
                    a = div.find('a', class_='hyperlink_appearance_undefined')
                    href = a['href']
                    title = a.text
                    div_info = div.find('div', class_='slQ-DR')
                    salary = '0'
                    p_salary = div_info.find('p')
                    if p_salary:
                        salary = p_salary.text
                    description = div_info('div', class_='PAM72f')#тут какой-то масив, по идее не должно быть такого['div: a: a: spin: ']
                    div_title = div.find('div', class_='L4BhzZ')
                    p_company = div_title.find('p', class_='z6WlhX')
                    company=''
                    if p_company:
                        company = p_company.text
                    img = div_title.find('img')
                    if img:
                        img_src = img['src']
                    else:
                        img_src =  'https://play-lh.googleusercontent.com/JCQ1opom-Kay8f3xVs9VfKmDKsKD3md5uKLJf93gsYAawE6UpzgN_2fALgS0mKOcNw=s256-rw'

                    jobs.append({'title': title, 'img_source': img_src, 'url': href,
                                 'description': description[0].text, 'company': company,
                                  'salary': salary, 'site': "jooble"})
            else:
                errors.append({'url': url, 'title': "Div does not exists"})
        else:
            errors.append({'url': url, 'title': "Page do not response"})

    return jobs, errors

In [11]:
def format_vacancy(job):
    salary = job.get('salary')  # Отримуємо значення поля "salary"
    description = job.get('description', '')  # Отримуємо опис вакансії, за замовчуванням - пустий рядок
    if salary and salary not in ['0', '']:  # Перевіряємо, чи є значення зарплати непустим числом
        vacancy_message = f"<b>{job['title']}</b>\n"\
                          f"<b>Компанія:</b> {job['company'].strip()}\n" \
                          f"<b>Зарплата:</b> {salary}\n"
    else:
        vacancy_message = f"<b>{job['title']}</b>\n"\
                          f"<b>Компанія:</b> {job['company'].strip()}\n"

    if description.strip():  # Перевіряємо, чи опис вакансії не є пустим рядком
        vacancy_message += f"<b>Опис вакансії:</b>\n{description.strip()}\n"

    vacancy_message += f"\nДетальніше за <a href='{job['url']}'>посиланням</a>"

    return vacancy_message

In [12]:
class STAGE(Enum):
  START = 0
  NEXT_PARAM_CITY = 1
  NEXT_PARAM_CATEGORY = 2
  NEXT_PARAM_POSITION = 3
  NEXT_PARAM_MIN_SALARY = 4
  NEXT_PARAM_EXPERIENCE = 5
  NEXT_PARAM_LANGUAGE = 6
  END_OF_PROFILE_FILLING = 7
  RECEIVING_VACANCIES = 8
  START_SEARCH = 9
  START_SEARCHING_FOR_NEW_VACANCIES = 10
  CREATE_REQUEST = 11 #Користувач хоче створити новий запит.
  MY_REQUESTS = 12 #Користувач хоче переглянути свої запити.
  PRICING = 13 # Користувач хоче ознайомитися з тарифами та оплатою.
  ABOUT_BOT = 14 #Користувач хоче дізнатися більше про бота.
  OFFER_VACANCY = 15 #Користувач хоче запропонувати свою вакансію.
  CONTACTS = 15 #Користувач хоче знайти контактну інформацію.


class Request:
  def __init__(self, language='', city='', position='', min_salary=0, experience=EXPERIENCE.DEFAULT_VALUE.value):
    self.language = language
    self.city = city
    self.position = position
    self.min_salary = int(min_salary)
    self.experience = int(experience)

  def __str__(self):
    return (f"Request(Language: {self.language}, City: {self.city}, "
            f"Position: {self.position}, Min Salary: {self.min_salary}, "
            f"Experience: {self.experience})")

  def set_language(self, language):
    self.language = language

  def set_city(self, city):
    self.city = city

  def set_position(self, position):
    self.position = position

  def set_min_salary(self, min_salary):
    self.min_salary = int(min_salary)

  def set_experience(self, experience):
    self.experience = int(experience)

In [13]:
def submit_vacancy(jobs, chat_id):
  for job in jobs:
    print(job)
    try:
        #print("job['img_source'] = ", job['img_source'])
        bot.send_photo(chat_id, job['img_source'], caption=format_vacancy(job), parse_mode='HTML')
    except ApiTelegramException as e:
        print(f"An error occurred while sending the photo: {e}. URL: {job['img_source']}. Skip vacancy.")


def start_seach_and_send_workua_vacancy(request:Request, chat_id, on_the_background:bool = False):
    if not on_the_background: days = 125 #вакансії за останній місяць
    else: days = 122 #вакансії за останній день
    url_work = build_url_workua(
            language=request.language,
            city=request.city, #ТУТ ПОХОДУ УКРАЇНСЬКОЮ НЕ МОЖНА, ТРЕБА ВИПРАВИТИ
            position=request.position,
            min_salary=request.min_salary,
            experience=request.experience,
            days=date
    )
    print("WORKUA URL: ", url_work)
    jobs_work, errors_work = work(url_work)
    print(jobs_work)
    print(errors_work)
    if not on_the_background:
      add_vacancies_to_db(request, chat_id, jobs_work, "workua")
      submit_vacancy(jobs_work, chat_id)
    else:
      return jobs_work, errors_work

def start_seach_and_send_dou_vacancy(request:Request, chat_id, on_the_background:bool = False):
    url_dou = build_url_dou(search=request.language, position=request.position, city=request.city, exp=request.experience ) #в доу можна скрапить і по 0-1 і по 1-3, тоже нужно решить проблему
    print("DOU URL: ", url_dou)
    jobs_dou, errors_dou = dou(url_dou)
    print(jobs_dou)
    print(errors_dou)
    if not on_the_background:
      add_vacancies_to_db(request, chat_id, jobs_dou, "dou")
      submit_vacancy(jobs_dou, chat_id)
    else:
      return jobs_dou, errors_dou

def start_seach_and_send_djinni_vacancy(request:Request, chat_id, on_the_background:bool = False):
    url_djinni = build_url_djinni(
      #all_keywords=request.language,
      primary_keyword=request.language,
      region="UKR",
      location=request.city,
      exp_level=request.experience,
      exp_rank=request.position,
      salary=convert_uah_to_usd_for_djinni(request.min_salary, 39.1),
      keywords=request.language
    )
    print("DJINNI URL: ", url_djinni)
    jobs_djinni, errors_djinni = djinni(url_djinni)
    add_vacancies_to_db(request, chat_id, jobs_djinni, "djinni")
    print(jobs_djinni)
    print(errors_djinni)
    if not on_the_background:
      submit_vacancy(jobs_djinni, chat_id)
    else:
      return jobs_djinni, errors_djinni

def start_seach_and_send_robota_vacancy(request:Request, chat_id, on_the_background:bool = False):
    url_robota=build_url_robota(language=request.language, city=request.city, position=request.position, salary = request.min_salary, experience=request.experience)
    print("ROBOTA URL: ", url_robota)
    jobs_rabota, errors_robota = robota(url_robota)
    print(jobs_rabota)
    print(errors_robota)
    if not on_the_background:
      add_vacancies_to_db(request, chat_id, jobs_rabota, "robota")
      submit_vacancy(jobs_rabota, chat_id)
    else:
      return jobs_rabota, errors_robota

def start_seach_and_send_jooble_vacancy(request:Request, chat_id, on_the_background:bool = False):
    if not on_the_background: date = 3 #вакансії за останню неділлю
    else: date = 8 #вакансії за останній день
    url_jooble = build_url_jooble(date=date,
                                  rgns=request.city,
                                  salaryMin=request.min_salary,
                                  ukw=request.position + ' ' + request.language,
                                  workExp=request.experience
                                 )
    print("JOOBLE URL: ", url_jooble)
    jobs_jooble, errors_jooble = jooble(url_jooble)
    print(jobs_jooble)
    print(errors_jooble)
    if not on_the_background:
      add_vacancies_to_db(request, chat_id, jobs_jooble, "jooble")
      submit_vacancy(jobs_jooble, chat_id)
    else:
      return jobs_jooble, errors_jooble

In [14]:
def check_db_has_such_vacancy(request_id, site, title, company):
    # Проверяем, существует ли уже такая вакансия в базе данных
    check_query = """
        SELECT COUNT(*) FROM vacancies
        WHERE request_id = %s AND site = %s AND title = %s AND company = %s
        """
    check_params = (request_id, site, title, company)
    check_result = execute_query(check_query, check_params)
    if check_result[0][0] == 0:
      return False
    else:
      return True

def add_NEW_vacancy_to_db(request_id, site, title, company, description):
    insert_query = """
                    INSERT INTO vacancies (request_id, site, title, company, description) VALUES(%s, %s, %s, %s, %s)
                    """
    insert_params = (request_id, site, title, company, description)
    insert_result = execute_query(insert_query, insert_params)
    return insert_result

def add_vacancies_to_db(request: Request, chat_id, vacancies, site):
    query = """
        SELECT request_id FROM requests
        WHERE chat_id = %s AND technology = %s AND position = %s AND city = %s AND min_salary = %s AND experience = %s
        """
    params = (chat_id, request.language, request.position, request.city, request.min_salary, request.experience)
    result = execute_query(query, params)

    if result:
        for vacancy in vacancies:#[:5]:  # Обрабатываем только первые 5 вакансий. НЕТ. ПОКА ЧТО будем добавлять ВСЕ вакансии в дб, чтобы потом не было проблем
            db_has_such_vacancy = check_db_has_such_vacancy(result[0][0], site, vacancy['title'], vacancy['company'].strip())
            # Если вакансия не найдена, добавляем её в базу данных
            if db_has_such_vacancy == False:  # Проверяем, что вакансия в бд не найдена
                insert_result = add_NEW_vacancy_to_db(result[0][0], site, vacancy['title'], vacancy['company'].strip(), vacancy.get('description', ''))
                print("Вакансия добавлена:", insert_result)
            else:
                print("Вакансия уже существует в базе данных")

In [15]:
!pip install schedule > null

In [16]:
import schedule
import time
import threading
from itertools import chain
keep_running = True

def get_requests_from_db():
    query = """
        SELECT * FROM requests
        """
    return execute_query(query)

print( get_requests_from_db())

def job(chat_id): #Тут змінимо функцію, щоб вона скрапила данні з сайту (якщо можливо фільтрувати, то фільтрувати по максимуму) і перевіряла чи є такі вакансії в базі данних і відсіювала такі
    print("Start working on background")
    requests_from_db = get_requests_from_db()
    for request_item in requests_from_db:
      request_from_db = Request(language=request_item[2], city=request_item[4], position=request_item[3], min_salary=request_item[5], experience=request_item[6])
      print(request_from_db)
      print("Start searching on background")
      jobs_work, errors_work = start_seach_and_send_workua_vacancy(request_from_db, 0, on_the_background = True)
      # jobs_dou, errors_dou = start_seach_and_send_dou_vacancy(request_from_db, 0, on_the_background = True)
      # jobs_djinni, errors_djinni = start_seach_and_send_djinni_vacancy(request_from_db, 0, on_the_background = True)
      # jobs_rabota, errors_robot = start_seach_and_send_robota_vacancy(request_from_db, 0, on_the_background = True)
      # jobs_jooble, errors_jooble = start_seach_and_send_jooble_vacancy(request_from_db, 0, on_the_background = True)
      print("End searching on background")

      vacancies = list(chain(jobs_work))#, jobs_dou, jobs_djinni, jobs_rabota, jobs_jooble))
      #vacancies = [jobs_work]#, jobs_dou, jobs_djinni, jobs_rabota, jobs_jooble]
      print("Start checking finding vacancies")
      for vacancy in vacancies:
        print("vacancy: ", vacancy)
        if check_db_has_such_vacancy(request_item[0], vacancy['site'], vacancy['title'], vacancy['company'].strip()) == False:
          print("Find new vacancy! Add to db and submit to chat")
          insert_result = add_NEW_vacancy_to_db(request_item[0], vacancy['site'], vacancy['title'], vacancy['company'].strip(), vacancy.get('description', ''))
          print("Вакансия добавлена:", insert_result)
          submit_vacancy([vacancy], int(request_item[1])) #request_item[1] - це chat_id
        else:
          print("This vacancy is already in the database")
          #bot.send_message(request_item[1], "Nothing new. I'm working...")
      print("End checking finding vacancies")

#job(869704769)

def start_scheduler():
    global keep_running
    keep_running = True

def stop_scheduler():
    global keep_running
    keep_running = False

def run_scheduler(chat_id):
    schedule.every(1).minutes.do(lambda: job(chat_id))
    global keep_running
    print("start run_scheduler. keep_running = ", keep_running)
    while keep_running:
        schedule.run_pending()
        time.sleep(1)

[(19, 869704769, 'Java', 'Junior', 'Kyiv', 0, -2), (20, 869704769, 'Java', 'Senior', 'Kyiv', 0, -2), (21, 869704769, 'Java', '', '', 0, 1), (22, 869704769, 'Android', 'Middle', '', 0, -2), (23, 869704769, 'Python', 'Trainee', 'Kyiv', 0, -2), (24, 535434829, 'Fullstack', '', 'Kyiv', 0, -2), (25, 535434829, 'C++', '', '', 0, -2), (26, 535434829, 'C#', '', '', 0, -2)]


In [17]:
def get_subscription_info(chat_id):
    result = execute_query("SELECT paid_subscription FROM users WHERE chat_id = %s", (chat_id,))
    if result:
        subscription_level = result[0][0]
        if subscription_level == 0:
            return "Ви ще не обрали жодного тарифу."
        elif subscription_level == 1:
            return "Ваш поточний тариф: 🔴 Економ."
        elif subscription_level == 2:
            return "Ваш поточний тариф: 🟠 Стандарт."
        elif subscription_level == 3:
            return "Ваш поточний тариф: 🟢 Бізнес."
    else:
        return "Ви ще не обрали жодного тарифу."


In [ ]:
from telebot import types
from telebot.apihelper import ApiTelegramException
from telebot.types import ReplyKeyboardMarkup, KeyboardButton
import threading
from datetime import datetime
from dateutil.relativedelta import relativedelta

TOKEN = '7161535193:AAG2EDGz-thMU4zFHRS0ZC3OoqnmGr9L79E'


bot = telebot.TeleBot(TOKEN)
request = Request()
stage = STAGE.START
print(stage)

def get_end_subscription_date():
    current_date = datetime.now().date()
    return current_date + relativedelta(months=1)

@bot.message_handler(commands=['pay1'])
def pay1_command(message):
    # Створення кнопки "Оплатити"
    markup = types.InlineKeyboardMarkup()
    markup.add(types.InlineKeyboardButton("Оплатити", callback_data='pay1_payment'))

    # Відправлення повідомлення з кнопкою
    bot.send_message(message.chat.id, """
    🔴 Економ
    Створюйте до двох(2) пошукових запитів вакансій на місяць.
    Тариф активується автоматично одразу після оплати та триває протягом одного календарного місяця.
    Оплачуючи Ви погоджуєтесь з Умовами Користування.
    """, reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data.startswith('pay1_payment'))
def pay1_payment_callback(call):
    end_date_subscription = get_end_subscription_date()
    execute_query("UPDATE users SET paid_subscription = 1, max_num_of_request = 2, end_date_subscription = %s WHERE chat_id = %s", (end_date_subscription, call.from_user.id))
    bot.send_message(call.message.chat.id, "Ваш поточний тариф змінено на Економ.")

@bot.message_handler(commands=['pay2'])
def pay2_command(message):
    # Створення кнопки "Оплатити"
    markup = types.InlineKeyboardMarkup()
    markup.add(types.InlineKeyboardButton("Оплатити", callback_data='pay2_payment'))
    bot.send_message(message.chat.id, """
    🟠 Стандарт
    Створюйте до чотирьох(4) пошукових запитів вакансій на місяць.
    Тариф активується автоматично одразу після оплати та триває протягом одного календарного місяця.
    Оплачуючи Ви погоджуєтесь з Умовами Користування.
    """, reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data.startswith('pay2_payment'))
def pay2_payment_callback(call):
    end_date_subscription = get_end_subscription_date()
    execute_query("UPDATE users SET paid_subscription = 2, max_num_of_request = 4, end_date_subscription = %s WHERE chat_id = %s", (end_date_subscription, call.from_user.id))
    bot.send_message(call.message.chat.id, "Ваш поточний тариф змінено на Стандарт.")

@bot.message_handler(commands=['pay3'])
def pay3_command(message):
    # Створення кнопки "Оплатити"
    markup = types.InlineKeyboardMarkup()
    markup.add(types.InlineKeyboardButton("Оплатити", callback_data='pay3_payment'))
    bot.send_message(message.chat.id, """
    🟢 Бізнес
    Створюйте до десяти(10) пошукових запитів вакансій на місяць.
    Тариф активується автоматично одразу після оплати та триває протягом одного календарного місяця.
    Оплачуючи Ви погоджуєтесь з Умовами Користування.
    """, reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data.startswith('pay3_payment'))
def pay3_payment_callback(call):
    end_date_subscription = get_end_subscription_date()
    execute_query("UPDATE users SET paid_subscription = 3, max_num_of_request = 10, end_date_subscription = %s WHERE chat_id = %s", (end_date_subscription, call.from_user.id))
    bot.send_message(call.message.chat.id, "Ваш поточний тариф змінено на Бізнес.")

@bot.message_handler(commands=['start'])
def start(message):
    global stage
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    # btn1 = types.KeyboardButton("Почати пошук роботи")
    # markup.add(btn1)
    bot.send_message(message.from_user.id, "👋 Привіт! Вас вітає бот з пошуку роботи!", reply_markup=markup)
    show_main_menu(message.from_user.id)
    stage = STAGE.START

def generate_keyboard(options):
    markup = ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for option in options:
        markup.add(KeyboardButton(option))
    return markup

cities = ["Повернутися в головне меню", "remote", "Kyiv", "Vinnytsia", "Dnipro", "Ivano-Frankivsk", "Zhytomyr", "Zaporizhzhia", "Lviv", "Mykolaiv", "Odesa", "Ternopil", "Kharkiv", "Khmelnytskyi", "Cherkasy", "Chernihiv", "Chernivtsi", "Uzhhorod"]
positions = ["Повернутися в головне меню", "Trainee","Intern" , "Junior", "Middle", "Senior", "Team Lead", "Chief of", "Head of"]

categories = ["Розробка", "Технічні", "Нетехнічні"]

programming_languages = ["Повернутися в головне меню", "JavaScript", "Front-End", "Fullstack", "Java", "C#", ".NET", "Python", "PHP", "Node.js", "iOS", "Android", "React Native", "C", "C++", "Embedded", "Flutter", "Golang", "Ruby", "Scala", "Salesforce", "Rust", "Elixir", "Kotlin", "ERP Systems"]
technical_specialties = ["Повернутися в головне меню", "QA Manual", "QA Automation", "Design", "2D/3D Artist", "Illustrator", "Gamedev", "Project Manager", "Product Manager", "Product Owner", "Delivery Manager", "Scrum Master", "Agile Coach", "Architect", "CTO", "DevOps", "Security", "Sysadmin", "Business Analyst", "Data Science", "Data Analyst", "Data Engineer", "SQL", "DBA", "Technical Writing"]
nontechnical_specialties = ["Повернутися в головне меню", "Marketing", "Sales", "Lead Generation", "SEO", "HR Recruiter", "Customer Support", "Technical Support", "Head", "Chief", "Finances"]

def show_create_request_menu(user_id):
    bot.send_message(user_id, "Оберіть параметр для пошуку:", reply_markup=generate_keyboard(["Категорія", "Місто", "Позиція", "Досвід роботи", "Зарплата", "Почати пошук!"]))

# def check_and_new_add_user_to_db(chat_id):
#     result = execute_query(f"SELECT * FROM users WHERE chat_id = {chat_id}")
#     if not result:
#         execute_query(f"INSERT INTO users (chat_id, paid_subscription) VALUES ({chat_id}, 0, 1, 1);")

def check_and_new_add_user_to_db(chat_id):
    # Проверяем, существует ли пользователь с указанным chat_id
    result = execute_query(f"SELECT * FROM users WHERE chat_id = {chat_id}")
    if not result:
      # Если пользователя нет в базе данных, добавляем его
      execute_query(f"INSERT INTO users (chat_id, paid_subscription, max_num_of_request, num_of_used_request) VALUES ({chat_id}, 0, 2, 0);")


    # else:
    #     # Получаем текущее значение num_of_used_request и max_num_of_request
    #     user_info = execute_query(f"SELECT num_of_used_request, max_num_of_request FROM users WHERE chat_id = {chat_id}")
    #     if user_info:
    #         num_of_used_request, max_num_of_request = user_info[0]
    #         # Проверяем, не превышает ли num_of_used_request max_num_of_request
    #         if num_of_used_request >= max_num_of_request:
    #             # Если превышает, выводим сообщение пользователю
    #             bot.send_message(message.from_user.id, "Вы исчерпали лимит запросов. Обратитесь за подпиской.")
    #         else:
    #             # Если пользователь не превысил максимальное количество запросов, увеличиваем значение num_of_used_request на 1
    #             execute_query(f"UPDATE users SET num_of_used_request = num_of_used_request + 1 WHERE chat_id = {chat_id}")


def add_request_to_db(chat_id, technology, position, city, min_salary, experience):
    # Проверяем, существует ли запрос с указанными параметрами
    existing_request = execute_query("SELECT * FROM requests WHERE chat_id = %s AND technology = %s AND position = %s AND city = %s AND min_salary = %s AND experience = %s", (chat_id, technology, position, city, min_salary, experience))
    if not existing_request:
                execute_query(f"UPDATE users SET num_of_used_request = num_of_used_request + 1 WHERE chat_id = {chat_id}")
                execute_query("INSERT INTO requests (chat_id, technology, position, city, min_salary, experience) VALUES (%s, %s, %s, %s, %s, %s)", (chat_id, technology, position, city, min_salary, experience))
    else:
      bot.send_message(chat_id, "Такий запит вже існує.")

def show_user_requests(chat_id):
    # Отримуємо всі запити користувача з бази даних за його chat_id
    query = "SELECT * FROM requests WHERE chat_id = %s"
    params = (chat_id,)
    user_requests = execute_query(query, params)

    if user_requests:
        # Якщо є запити, виводимо їх у боті
         for i, req in enumerate(user_requests, start=1):
            request_text = f"""
            Номер запиту : {i}
            Категорія: {req[1]}
            Позиція: {req[2]}
            Місто: {req[3]}
            Мінімальна заробітна плата: {req[4]}
            Досвід роботи: {req[5]}
            """
            bot.send_message(chat_id, request_text)

    else:
        bot.send_message(chat_id, "Ви ще не створили жодного запиту.")


@bot.message_handler(content_types=['text'])
def handle_text(message):
    global stage
    threading.Thread(target=check_and_new_add_user_to_db, args=(message.from_user.id, )).start()

    if message.text == "Почати пошук!" and stage==STAGE.CREATE_REQUEST:
        if request.language == "":
          bot.send_message(message.from_user.id, "Будь ласка, вкажіть категорію для пошуку (мову програмування)!")
          bot.send_message(message.from_user.id, "Оберіть категорію:", reply_markup=generate_keyboard(categories))
          stage = STAGE.NEXT_PARAM_CATEGORY
        else:
          stage = STAGE.START_SEARCH
    if stage == STAGE.START:
        show_main_menu(message.from_user.id)
        if message.text == "Мої запити":
            stage = STAGE.MY_REQUESTS
            show_user_requests(message.from_user.id)
            stage = STAGE.START
        elif message.text == "Створити запит":
            show_create_request_menu(message.from_user.id)
            stage = STAGE.CREATE_REQUEST
        elif message.text == "Тарифи та оплата":
           subscription_info = get_subscription_info(message.from_user.id)
           bot.send_message(message.from_user.id, f"""
                        Ваш поточний тариф: {subscription_info}

  Хочете мати більше можливостей у використанні нашого бота? Оберіть один з доступних тарифів:

  🔴 Економ
  - Дозволяє створювати до двох запитів для пошуку вакансій на місяць
  - Ціна: $1 на місяць
  - Для замовлення тарифу використайте команду /pay1

  🟠 Стандарт
  - Дозволяє створювати до чотирьох запитів для пошуку вакансій на місяць
  - Ціна: $2 на місяць
  - Для замовлення тарифу використайте команду /pay2

  🟢 Бізнес
  - Дозволяє створювати до десяти запитів для пошуку вакансій на місяць
  - Ціна: $5 на місяць
  - Для замовлення тарифу використайте команду /pay3

  Оберіть підходящий для вас тариф, щоб отримати доступ до більшого обсягу вакансій та інших корисних функцій.
  """)
           stage = STAGE.PRICING
                # Показати інформацію про тарифи та оплату
        elif message.text == "Про бот":
            stage = STAGE.ABOUT_BOT
                # Показати інформацію про бота
        elif message.text == "Запропонувати свою вакансію":
            stage = STAGE.OFFER_VACANCY
                # Виконати дії для пропозиції вакансії
        elif message.text == "Контакти":
            stage = STAGE.CONTACTS
        # Показати контактну інформацію
    elif stage == STAGE.CREATE_REQUEST:
        if message.text == "Категорія":
            bot.send_message(message.from_user.id, "Оберіть категорію:", reply_markup=generate_keyboard(categories))
            stage = STAGE.NEXT_PARAM_CATEGORY
        elif message.text == "Місто":
            bot.send_message(message.from_user.id, "Виберіть місто для пошуку:", reply_markup=generate_keyboard(cities))
            stage = STAGE.NEXT_PARAM_CITY
        elif message.text == "Позиція":
            bot.send_message(message.from_user.id, "Виберіть вашу позицію:", reply_markup=generate_keyboard(positions))
            stage = STAGE.NEXT_PARAM_POSITION
        elif message.text == "Досвід роботи":
            bot.send_message(message.from_user.id, "Вкажіть кількість років досвіду:")
            stage = STAGE.NEXT_PARAM_EXPERIENCE
        elif message.text == "Зарплата":
            bot.send_message(message.from_user.id, "Вкажіть мінімальну зарплату (грн):")
            stage = STAGE.NEXT_PARAM_MIN_SALARY
    elif stage == STAGE.NEXT_PARAM_CATEGORY:
        if message.text == "Розробка":
            bot.send_message(message.from_user.id, "Оберіть мову програмування для пошуку:", reply_markup=generate_keyboard(programming_languages))
            stage = STAGE.NEXT_PARAM_LANGUAGE
        elif message.text == "Технічні":
            bot.send_message(message.from_user.id, "Оберіть технічну спеціальність для пошуку:", reply_markup=generate_keyboard(technical_specialties))
            stage = STAGE.NEXT_PARAM_LANGUAGE
        elif message.text == "Нетехнічні":
            bot.send_message(message.from_user.id, "Оберіть нетехнічну спеціальність для пошуку:", reply_markup=generate_keyboard(nontechnical_specialties))
            stage = STAGE.NEXT_PARAM_LANGUAGE

    elif stage == STAGE.NEXT_PARAM_LANGUAGE:
        if message.text == "Повернутися в головне меню":
            stage = STAGE.START
            show_main_menu(message.from_user.id)
        else:
            request.set_language(message.text)
            stage = STAGE.CREATE_REQUEST
            show_create_request_menu(message.from_user.id)
    elif stage == STAGE.NEXT_PARAM_CITY:
        if message.text == "Повернутися в головне меню":
            stage = STAGE.START
            show_main_menu(message.from_user.id)
        else:
            request.set_city(message.text)
            stage = STAGE.CREATE_REQUEST
            show_create_request_menu(message.from_user.id)
    elif stage == STAGE.NEXT_PARAM_POSITION:
        if message.text == "Повернутися в головне меню":
            stage = STAGE.START
            show_main_menu(message.from_user.id)
        else:
            request.set_position(message.text)
            stage = STAGE.CREATE_REQUEST
            show_create_request_menu(message.from_user.id)
    elif stage == STAGE.NEXT_PARAM_EXPERIENCE:
        if message.text == "Повернутися в головне меню":
            stage = STAGE.START
            show_main_menu(message.from_user.id)
        else:
            request.set_experience(message.text)
            stage = STAGE.CREATE_REQUEST
            show_create_request_menu(message.from_user.id)
    elif stage == STAGE.NEXT_PARAM_MIN_SALARY:
        if message.text == "Повернутися в головне меню":
            stage = STAGE.START
            show_main_menu(message.from_user.id)
        else:
            request.set_min_salary(message.text)
            stage = STAGE.CREATE_REQUEST
            show_create_request_menu(message.from_user.id)

    elif stage == STAGE.START_SEARCH:
        show_main_menu(message.from_user.id)
        stop_scheduler()
        threads = []
        #threads.append(threading.Thread(target=start_seach_and_send_workua_vacancy, args=(request, message.from_user.id)))
        #threads.append(threading.Thread(target=start_seach_and_send_dou_vacancy, args=(request, message.from_user.id)))
        #threads.append(threading.Thread(target=start_seach_and_send_djinni_vacancy, args=(request, message.from_user.id)))
        #threads.append(threading.Thread(target=start_seach_and_send_robota_vacancy, args=(request, message.from_user.id)))
        threads.append(threading.Thread(target=start_seach_and_send_jooble_vacancy, args=(request, message.from_user.id)))

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()
        #stage = STAGE.START_SEARCHING_FOR_NEW_VACANCIES # так не можна буде потім змінювати параметри пошуку, треба ще подумати
        stage = STAGE.START

        start_scheduler()
        threading.Thread(target=run_scheduler, args=(message.from_user.id, )).start()
        #Придумать что-то, чтобы после перезапуска бота это автоматически запускалось
        threading.Thread(target=add_request_to_db, args=(message.from_user.id,request.language, request.position, request.city, request.min_salary, request.experience)).start()
    else:
        bot.send_message(message.from_user.id, "Спробуйте ввести іншу команду. Введіть /start.") #TODO: кудись перемістити, ця частина не працює



# Функція для відображення головного меню
def show_main_menu(user_id):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    item1 = types.KeyboardButton("Створити запит")
    item2 = types.KeyboardButton("Мої запити")
    item3 = types.KeyboardButton("Тарифи та оплата")
    item4 = types.KeyboardButton("Про бот")
    item5 = types.KeyboardButton("Запропонувати свою вакансію")
    item6 = types.KeyboardButton("Контакти")
    markup.add(item1, item2, item3, item4, item5, item6)
    #bot.send_message(user_id, "Оберіть дію:", reply_markup=markup)


bot.polling(none_stop=True, interval=0)

STAGE.START


In [ ]:
stop_scheduler()

In [ ]:
# ЗАДАЧІ:
# 1. Залить на гитхаб рабочую версию
# 2. Зробити видалення своїх вакансій за допомогою кнопки під вакансією
# 3. Перевірка на кількість реквестів, щоб більше ніж доступно не можна було зробити
# 4. В цілому зробити більш гарний інтерфейс
# 5. Зробити шоб після перезапуска фоновий пошук вакансій запускався автоматично, а не після того, як ми вручну додали новий запит
# ЗРОБЛЕНО 6. Перевірити як інші сайти працюють з "С++", "С#", якщо є проблеми - швидко виправити аналогічно до workua
# 7. Спробувати запустити бот локально в PyCharm і перевірити як швидко буде працювати з запитами до БД
# 8. Реалізувати допоміжні функції для robota і djini
# 9. Зробити, щоб виводилось по 10 запитів, а інші щоб після натискання кнопки ЩЕ
# 10. Зробити функцінал для додавання унікальних вакансій, які є лише в боті
# 11. Додати адміна, мабуть зробити для нього трохи інший інтерфейс


# . Рефакторинг коду, створити класи, щоб все гарно виглядало - можна зробити на будь якому етапі, бажано якомога раніше
# . Можливо ще подумати з приводу оплати тарифів грошима
# . Задеплоїти систему (optional), але бажано